In [2]:
import zipfile
import os

# Path to the uploaded ZIP file (It will be in /content)
zip_path = 'dataset.zip'  # Replace with your uploaded file name
extract_path = 'dataset'  # Directory to extract to

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted successfully!")

Dataset extracted successfully!


Verify the Dataset Folder Structure

In [8]:
# Update the dataset directory path
dataset_dir = 'dataset/dataset'  # Adjusted path

# List the poets' folders
print("Poets Folder:", os.listdir(dataset_dir))

# Pick the first poet's folder
sample_poet = os.listdir(dataset_dir)[0]
sample_poet_path = os.path.join(dataset_dir, sample_poet)

# List the subfolders inside the poet's folder (should be 'en', 'ur', 'hi')
print(f"Subfolders for {sample_poet}: {os.listdir(sample_poet_path)}")


Poets Folder: ['.DS_Store', 'ahmad-faraz', 'akbar-allahabadi', 'allama-iqbal', 'altaf-hussain-hali', 'ameer-khusrau', 'bahadur-shah-zafar', 'dagh-dehlvi', 'fahmida-riaz', 'faiz-ahmad-faiz', 'firaq-gorakhpuri', 'gulzar', 'habib-jalib', 'jaan-nisar-akhtar', 'jaun-eliya', 'javed-akhtar', 'jigar-moradabadi', 'kaifi-azmi', 'meer-anees', 'meer-taqi-meer', 'mirza-ghalib', 'mohsin-naqvi', 'naji-shakir', 'naseer-turabi', 'nazm-tabatabai', 'nida-fazli', 'noon-meem-rashid', 'parveen-shakir', 'sahir-ludhianvi', 'wali-mohammad-wali', 'waseem-barelvi']


NotADirectoryError: [WinError 267] The directory name is invalid: 'dataset/dataset\\.DS_Store'

Load & Read the Poetry Files from "en" Folders
Now, we need to extract poetry only from the "en" folder of each poet.

In [7]:
import os

# Path to the dataset containing poet folders
dataset_dir = 'dataset/dataset'

# List to store all poetry texts
poetry_texts = []

# Iterate through each poet's folder
for poet in os.listdir(dataset_dir):
    poet_path = os.path.join(dataset_dir, poet)

    # Check if it's a directory (ignore files like .DS_Store)
    if os.path.isdir(poet_path):
        en_path = os.path.join(poet_path, "en")  # Path to "en" poetry folder

        # Check if "en" folder exists
        if os.path.exists(en_path):
            for file in os.listdir(en_path):
                file_path = os.path.join(en_path, file)

                # Read the poetry file and append to list
                with open(file_path, "r", encoding="utf-8") as f:
                    poetry_texts.append(f.read())

# Print sample poetry to verify
print(f"Total Poems Collected: {len(poetry_texts)}")
print("Sample Poem:\n", poetry_texts[0][:500])  # Print first 500 characters


Total Poems Collected: 1314
Sample Poem:
 
aañkh se duur na ho dil se utar jā.egā 
vaqt kā kyā hai guzartā hai guzar jā.egā 
itnā mānūs na ho ḳhalvat-e-ġham se apnī 
tū kabhī ḳhud ko bhī dekhegā to Dar jā.egā 
Dūbte Dūbte kashtī ko uchhālā de duuñ 
maiñ nahīñ koī to sāhil pe utar jā.egā 
zindagī terī atā hai to ye jaane vaalā 
terī baḳhshish tirī dahlīz pe dhar jā.egā 
zabt lāzim hai magar dukh hai qayāmat kā 'farāz' 
zālim ab ke bhī na ro.egā to mar jā.egā 


Preprocess the Text Data
Before feeding data into the model, we need to:

Lowercase the text (optional)
Remove unwanted characters & extra spaces
Tokenize the text into sequences

In [17]:
import re

def preprocess_text(text):
    # Remove special characters except basic punctuations
    # text = re.sub(r'[^a-zA-Z0-9.,!?\'\s]', '', text)

    # Convert multiple spaces into a single space
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Apply preprocessing to all poems
cleaned_poetry = [preprocess_text(poem) for poem in poetry_texts]

# Print a cleaned sample
print("Cleaned Sample:\n", cleaned_poetry[0][:500])


Cleaned Sample:
 shahīd-e-ishq hue qais nāmvar kī tarah jahāñ meñ aib bhī ham ne kiye hunar kī tarah kuchh aaj shaam se chehra hai faq sahar kī tarah Dhalā hī jaatā huuñ furqat meñ dopahar kī tarah siyāh-baḳhtoñ ko yuuñ baaġh se nikāl ai charḳh ki chaar phuul to dāman meñ hoñ sipar kī tarah tamām ḳhalq hai ḳhvāhān-e-ābrū ai rab chhupā mujhe sadaf-e-qabr meñ guhar kī tarah tujhī ko dekhūñgā jab tak haiñ barqarār āñkheñ mirī nazar na phiregī tirī nazar kī tarah hamārī qabr pe kyā ehtiyāj-e-ambar-o-ūd sulag rahā ha


Convert Poetry into Sequences
Since LSTM models require numerical input, we need to:

Tokenize words into unique integers
Create input-output sequences (X: previous words, Y: next word)

In [5]:
!pip install torch==2.0.1 torchtext==0.15.2 --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 332.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 198.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 210.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 216.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 251.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 201.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 241.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 248.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 180.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 224.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 249.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, DataLoader

# Tokenize poetry into words
def tokenize_poetry(poetry_list):
    return [poem.split() for poem in poetry_list]

# Build vocabulary
tokenized_poems = tokenize_poetry(cleaned_poetry)
vocab = build_vocab_from_iterator(tokenized_poems, specials=["<PAD>", "<UNK>"])
vocab.set_default_index(vocab["<UNK>"])

# Convert words to indices
def encode_poetry(poem):
    return [vocab[word] for word in poem]

encoded_poems = [encode_poetry(poem) for poem in tokenized_poems]

# Print vocabulary size
print(f"Vocabulary Size: {len(vocab)}")
print("Encoded Sample Poem:", encoded_poems[0][:10])  # Show first 10 encoded words


Vocabulary Size: 17344
Encoded Sample Poem: [14417, 79, 950, 12695, 6, 93, 118, 4, 1082, 10]


Prepare Data for Training
We need to:

Create sequences of words (e.g., first 5 words → predict next word)
Pad sequences for uniform input size
Split data into training & validation

In [19]:
from torch.nn.utils.rnn import pad_sequence

class PoetryDataset(Dataset):
    def __init__(self, poems, seq_length=5):
        self.seq_length = seq_length
        self.data = []

        # Create sequences
        for poem in poems:
            if len(poem) > seq_length:
                for i in range(len(poem) - seq_length):
                    seq = poem[i:i+seq_length]
                    target = poem[i+seq_length]
                    self.data.append((seq, target))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sequence, target = self.data[idx]
        return torch.tensor(sequence), torch.tensor(target)

# Create dataset
seq_length = 5
dataset = PoetryDataset(encoded_poems, seq_length)

# DataLoader
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Print sample batch
for x, y in dataloader:
    print("Sample Input:", x[0])
    print("Sample Target:", y[0])
    break


Sample Input: tensor([ 14, 271,  15,   2,  88])
Sample Target: tensor(5141)


Build an LSTM Model in PyTorch

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim

class PoetryLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_layers=2):
        super(PoetryLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, batch_first=True)

        # Layer Normalization (Added)
        self.layer_norm = nn.LayerNorm(hidden_dim)

        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)

        # Apply Layer Normalization
        lstm_out = self.layer_norm(lstm_out)

        out = self.fc(lstm_out[:, -1])  # Use last LSTM output
        return out

# Model
model = PoetryLSTM(vocab_size=len(vocab)).to("cuda")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

# Learning Rate Scheduler (Reduce LR after 25 epochs)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=25, gamma=0.1)

print("Model Initialized")


Model Initialized


Train the Model

In [22]:
num_epochs = 51

for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for x_batch, y_batch in dataloader:
        x_batch, y_batch = x_batch.to("cuda"), y_batch.to("cuda")

        optimizer.zero_grad()
        output = model(x_batch)

        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.detach().item()  # Detach to save memory

    # Reduce learning rate after step_size epochs
    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader):.4f}, LR: {scheduler.get_last_lr()[0]:.6f}")

print("Training Completed!")


Epoch [1/51], Loss: 6.9408, LR: 0.000500
Epoch [2/51], Loss: 6.0723, LR: 0.000500
Epoch [3/51], Loss: 5.3268, LR: 0.000500
Epoch [4/51], Loss: 4.5049, LR: 0.000500
Epoch [5/51], Loss: 3.6548, LR: 0.000500
Epoch [6/51], Loss: 2.9274, LR: 0.000500
Epoch [7/51], Loss: 2.3332, LR: 0.000500
Epoch [8/51], Loss: 1.8382, LR: 0.000500
Epoch [9/51], Loss: 1.4150, LR: 0.000500
Epoch [10/51], Loss: 1.0740, LR: 0.000500
Epoch [11/51], Loss: 0.8096, LR: 0.000500
Epoch [12/51], Loss: 0.6206, LR: 0.000500
Epoch [13/51], Loss: 0.4935, LR: 0.000500
Epoch [14/51], Loss: 0.4195, LR: 0.000500
Epoch [15/51], Loss: 0.3781, LR: 0.000500
Epoch [16/51], Loss: 0.3448, LR: 0.000500
Epoch [17/51], Loss: 0.3219, LR: 0.000500
Epoch [18/51], Loss: 0.3011, LR: 0.000500
Epoch [19/51], Loss: 0.2874, LR: 0.000500
Epoch [20/51], Loss: 0.2840, LR: 0.000500
Epoch [21/51], Loss: 0.2617, LR: 0.000500
Epoch [22/51], Loss: 0.2540, LR: 0.000500
Epoch [23/51], Loss: 0.2502, LR: 0.000500
Epoch [24/51], Loss: 0.2353, LR: 0.000500
E

In [29]:
import random

def generate_poetry(seed_text, model, vocab, max_words=40):
    model.eval()
    words = seed_text.split()

    for _ in range(max_words):
        encoded = torch.tensor([vocab[word] for word in words[-5:]]).unsqueeze(0).to("cuda")
        with torch.no_grad():
            output = model(encoded)
            next_word = vocab.lookup_token(output.argmax().item())
            words.append(next_word)

    return " ".join(words)

# Example Usage
seed = "ajab junūn-e-masāfat meñ ghar se"
print("Generated Poetry:", generate_poetry(seed, model, vocab))


Generated Poetry: ajab junūn-e-masāfat meñ ghar se niklā thā ḳhabar nahīñ hai ki sūraj kidhar se niklā thā ye kaun phir se unhīñ rāstoñ meñ chhoḌ gayā abhī abhī to azāb-e-safar se niklā thā ye tiir dil meñ magar be-sabab nahīñ utrā koī to harf lab-e-chāragar se


In [24]:
# Save the trained model
torch.save(model.state_dict(), "poetry_lstm_model.pth")
import shutil

# Move the model to a location that can be downloaded
shutil.move("poetry_lstm_model.pth", "/mnt/data/poetry_lstm_model.pth")
from google.colab import files
files.download("/mnt/data/poetry_lstm_model.pth")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/poetry_lstm_model.pth'